In [1]:
import pandas as pd
import bar_chart_race as bcr
import numpy as np
from bs4 import BeautifulSoup
from sjvisualizer import DataHandler, Canvas, BarRace, StaticImage, StaticText
import requests
import re
import warnings
from PIL import Image
import os
import json
warnings.filterwarnings("ignore")

In [2]:
# Dicionários úteis
nacionalidades_pais = {
    'Italian': 'Itália', 'British': 'Reino Unido', 'French': 'França', 'Belgian': 'Bélgica',
    'Argentine': 'Argentina', 'Irish': 'Irlanda', 'Thai': 'Tailândia', 'Swiss': 'Suíça',
    'Monegasque': 'Mônaco', 'American': 'Estados Unidos', 'German': 'Alemanha',
    'Brazilian': 'Brasil', 'Spanish': 'Espanha', 'Australian': 'Austrália', 'Uruguayan': 'Uruguai',
    'East German': 'Alemanha', 'Dutch': 'Países Baixos', 'Swedish': 'Suécia',
    'Argentine-Italian': 'Argentina', 'New Zealander': 'Nova Zelândia', 'Portuguese': 'Portugal',
    'Venezuelan': 'Venezuela', 'American-Italian': 'Estados Unidos', 'South African': 'África do Sul',
    'Mexican': 'México', 'Canadian': 'Canadá', 'Rhodesian': 'Rodésia', 'Austrian': 'Áustria',
    'Liechtensteiner': 'Liechtenstein', 'Danish': 'Dinamarca', 'Finnish': 'Finlândia',
    'Japanese': 'Japão', 'Chilean': 'Chile', 'Colombian': 'Colômbia', 'Czech': 'República Tcheca',
    'Malaysian': 'Malásia', 'Hungarian': 'Hungria', 'Indian': 'Índia', 'Polish': 'Polônia',
    'Russian': 'Rússia', 'Indonesian': 'Indonésia'
}

codigos_paises = {
    "ad": "Andorra", "ae": "Emirados Árabes Unidos", "ar": "Argentina", "at": "Áustria",
    "au": "Austrália", "be": "Bélgica", "ba": "Bósnia e Herzegovina", "br": "Brasil", "bg": "Bulgária",
    "ca": "Canadá", "ch": "Suíça", "cl": "Chile", "cn": "China", "co": "Colômbia",
    "cr": "Costa Rica", "cz": "República Tcheca", "de": "Alemanha", "dk": "Dinamarca",
    "ec": "Equador", "ee": "Estônia", "eg": "Egito", "es": "Espanha", "fi": "Finlândia",
    "fr": "França", "gb": "Reino Unido", "gr": "Grécia", "hk": "Hong Kong", "hr": "Croácia",
    "hu": "Hungria", "id": "Indonésia", "ie": "Irlanda", "il": "Israel", "in": "Índia",
    "is": "Islândia", "it": "Itália", "jp": "Japão", "kr": "Coreia do Sul", "kw": "Kuwait",
    "kz": "Cazaquistão", "lb": "Líbano", "lt": "Lituânia", "lu": "Luxemburgo", "lv": "Letônia",
    "mc": "Mônaco", "mx": "México", "my": "Malásia", "nl": "Países Baixos", "no": "Noruega",
    "nz": "Nova Zelândia", "pe": "Peru", "ph": "Filipinas", "pl": "Polônia", "pt": "Portugal",
    "qa": "Catar", "ro": "Romênia", "ru": "Rússia", "sa": "Arábia Saudita", "se": "Suécia",
    "sg": "Singapura", "si": "Eslovênia", "sk": "Eslováquia", "sm": "San Marino", "th": "Tailândia",
    "tn": "Tunísia", "tr": "Turquia", "ua": "Ucrânia","us": "Estados Unidos", "uy": "Uruguai",
    "ve": "Venezuela",
}

# Leitura dos dados

Os dados foram baixados da planilha dispinível neste [link](https://docs.google.com/spreadsheets/d/1Avshw_wXQ7XX3V8iTfKIvx8JxjYulMtP/edit#gid=1317792570).

In [3]:
# Ajustando URL do Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/1Avshw_wXQ7XX3V8iTfKIvx8JxjYulMtP/edit#gid=1317792570"
url = sheet_url.replace("/edit#gid=", "/export?format=csv&gid=")

In [4]:
df = pd.read_csv(url)
df.head()

,season,round,raceName,date,circuitId,circuitName,number,position,positionText,points,...,dateOfBirth,nationality,constructorId,name,nationality.1,results.grid,results.laps,results.status,Circ Count,Full Name
0,1950,1,British Grand Prix,1950-05-13,silverstone,Silverstone Circuit,2.0,1,1,9.0,...,1906-10-30,Italian,alfa,Alfa Romeo,Italian,1,70,Finished,Silverstone Circuit,Nino Farina
1,1950,1,British Grand Prix,1950-05-13,silverstone,Silverstone Circuit,3.0,2,2,6.0,...,1898-06-09,Italian,alfa,Alfa Romeo,Italian,2,70,Finished,NaN,Luigi Fagioli
2,1950,1,British Grand Prix,1950-05-13,silverstone,Silverstone Circuit,4.0,3,3,4.0,...,1911-07-02,British,alfa,Alfa Romeo,Italian,4,70,Finished,NaN,Reg Parnell
3,1950,1,British Grand Prix,1950-05-13,silverstone,Silverstone Circuit,14.0,4,4,3.0,...,1904-10-08,French,lago,Talbot-Lago,French,6,68,+2 Laps,NaN,Yves Cabantous
4,1950,1,British Grand Prix,1950-05-13,silverstone,Silverstone Circuit,15.0,5,5,2.0,...,1905-11-05,French,lago,Talbot-Lago,French,9,68,+2 Laps,NaN,Louis Rosier


In [5]:
# Informações sobre colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24617 entries, 0 to 24616
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season          24617 non-null  int64  
 1   round           24617 non-null  int64  
 2   raceName        24617 non-null  object 
 3   date            24617 non-null  object 
 4   circuitId       24617 non-null  object 
 5   circuitName     24617 non-null  object 
 6   number          24611 non-null  float64
 7   position        24617 non-null  int64  
 8   positionText    24617 non-null  object 
 9   points          24617 non-null  float64
 10  driverId        24617 non-null  object 
 11  givenName       24617 non-null  object 
 12  familyName      24617 non-null  object 
 13  dateOfBirth     24616 non-null  object 
 14  nationality     24617 non-null  object 
 15  constructorId   24617 non-null  object 
 16  name            24617 non-null  object 
 17  nationality.1   24617 non-null 

# Limpeza

In [6]:
# Selecionando as colunas que serão utilizadas
my_cols = ["date", "position", "Full Name", "nationality"]
df = df[my_cols]

In [7]:
# Conversão da string da data para o tipo datetime
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
# df["date"] = pd.DatetimeIndex(df["date"]).year

In [8]:
# Mantém apenas os vencedores
df = df[df["position"] == 1]

In [9]:
# Criando a coluna país
df["country"] = df['nationality'].apply(lambda x: nacionalidades_pais.get(x, ''))
df = df.drop(columns="nationality")
df.tail()

,date,position,Full Name,country
24517,2019-10-13,1,Valtteri Bottas,Finlândia
24537,2019-10-27,1,Lewis Hamilton,Reino Unido
24557,2019-11-03,1,Valtteri Bottas,Finlândia
24577,2019-11-17,1,Max Verstappen,Países Baixos
24597,2019-12-01,1,Lewis Hamilton,Reino Unido


# Atualizando os dados

Como temos apenas dados até o fim da temporada 2019, utilizamos o site https://www.racing-statistics.com para conseguir o restante dos dados relevantes

In [10]:
def dados_web(ano: int) -> pd.DataFrame:
    # Site
    url = f"https://www.racing-statistics.com/en/seasons/{ano}"
    # Request do HTML da página
    html = requests.get(url)
    # Encontrando a tabela de vencedores por corrida
    soup = BeautifulSoup(html.content, "html.parser")
    table = soup.find_all("table")[1]
    # Extração dos dados
    linhas = table.find_all("tr")
    dados = []
    for i, linha in enumerate(linhas):
        colunas = linha.find_all("td")
        if i != 0:
            # Extraindo o país
            codigo_pais = re.search(r"flag-icon-([a-z]{2})\s*",
                                    colunas[4].find("span").get("class")[1]).group(1)
            nome_pais = codigos_paises.get(codigo_pais, "")
            # Colocando na lista de dados
            colunas = [nome_pais] + [col.text.strip() for col in colunas]
            dados.append(colunas)
    # Criando pandas dataframe
    df = pd.DataFrame(dados)
    # Cria coluna de posições
    df["position"] = 1
    # Seleciona apenas as colunas desejadas
    df = df[[2, "position", 6, 0]]
    # Renomeando colunas 
    df = df.rename(columns={2: "date", 6: "Full Name", 0: "country"})
    # Conversão da data
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
#     df["date"] = pd.DatetimeIndex(df["date"]).year
    return df

In [11]:
# Inserindo os anos a partir de 2019 no dataframe
for ano in range(2020, 2024):
    dados = dados_web(ano)
    df = pd.concat([df, dados], axis=0, ignore_index=True)
# Inserindo a corrida do último domingo (30/04)
dados = pd.DataFrame({"date": [pd.to_datetime("2023-04-30")],
                      "position": [1], "Full Name": ["Sergio Pérez"],
                      "country": ["México"]})
df = pd.concat([df, dados], axis=0, ignore_index=True)

In [12]:
df.tail()

,date,position,Full Name,country
1087,2022-11-20,1,Max Verstappen,Países Baixos
1088,2023-03-05,1,Max Verstappen,Países Baixos
1089,2023-03-19,1,Sergio Pérez,México
1090,2023-04-02,1,Max Verstappen,Países Baixos
1091,2023-04-30,1,Sergio Pérez,México


# Formatação para uso do bar chart race

## Vitórias por piloto

In [13]:
# Formatação para que cada piloto tenha uma coluna
df_pilotos = df.pivot_table(values="position", columns="Full Name", index="date", aggfunc="sum")
df_pilotos = df_pilotos.fillna(0)

In [14]:
df_pilotos.tail()

Full Name,Alain Prost,Alan Jones,Alberto Ascari,Alessandro Nannini,Ayrton Senna,Bill Vukovich,Bob Sweikert,Bruce McLaren,Carlos Pace,Carlos Reutemann,...,Sam Hanks,Sebastian Vettel,Sergio Pérez,Stirling Moss,Thierry Boutsen,Tony Brooks,Troy Ruttman,Valtteri Bottas,Vittorio Brambilla,Wolfgang von Trips
date,,,,,,,,,,,,,,,,,,,,,
2022-11-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Realiza a soma das vitórias dos pilotos ao longo do tempo
df_cumulativo_pilotos = df_pilotos.cumsum()

### Bandeiras por piloto

In [16]:
bandeira_pilotos = dict(zip(df["Full Name"], df["country"]))
pasta="assets"
for piloto in bandeira_pilotos.keys():
    # Nacionalidade
    pais = bandeira_pilotos[piloto]
    # Bandeira
    f = os.path.join(pasta, f"{pais}.png")
    bandeira = Image.open(f)
    # Cópia com nome do corredor
    bandeira.save(os.path.join(pasta, f"{piloto}.png"))

## Vitórias por país

In [17]:
# Formatação para que cada país tenha uma coluna
df_paises = df.pivot_table(values="position", columns="country", index="date", aggfunc="sum")
df_paises = df_paises.fillna(0)

In [18]:
df_paises.tail()

country,Alemanha,Argentina,Austrália,Brasil,Bélgica,Canadá,Colômbia,Espanha,Estados Unidos,Finlândia,...,Mônaco,Nova Zelândia,Países Baixos,Polônia,Reino Unido,Suécia,Suíça,Venezuela,África do Sul,Áustria
date,,,,,,,,,,,,,,,,,,,,,
2022-11-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-04-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Realiza a soma das vitórias dos pilotos de cada país ao longo do tempo
df_cumulativo_paises = df_paises.cumsum()

# Criação da bar chart race

In [20]:
canvas = Canvas.canvas()

## Vitórias por piloto

In [21]:
bar_chart1 = BarRace.bar_race(df=df_cumulativo_pilotos, canvas=canvas.canvas,
                             width=800, height=700, x_pos=200-40, y_pos=210)

In [22]:
canvas.add_sub_plot(bar_chart1)
canvas.add_title("Maiores vencedores da Fórmula 1 (1° Lugar)")
canvas.add_sub_title("1950 - Presente", color=tuple([128] * 3))
canvas.add_time(df=df_cumulativo_pilotos, time_indicator="year")

## Vitórias por país

In [23]:
# Cores dos países
with open("cores-paises.json", encoding="utf-8") as f:
    cores = json.load(f)

In [24]:
# Bar chart países
bar_chart2 = BarRace.bar_race(df=df_cumulativo_paises, canvas=canvas.canvas,
                             width=800, height=700, x_pos=1000+60, y_pos=210,
                             colors=cores)

In [25]:
canvas.add_sub_plot(bar_chart2)

In [26]:
# Imagens
logo = StaticImage.static_image(canvas=canvas.canvas, file="assets/logo_f1_transparente.png")
carro = StaticImage.static_image(canvas=canvas.canvas, file="assets/car.png", 
                                 y_pos=900, x_pos=860, width=200, height=200)
canvas.add_sub_plot(logo)
canvas.add_sub_plot(carro)

In [27]:
# Textos
pilotos = StaticText.static_text(canvas=canvas.canvas, text="Pilotos",
                                 height=35, y_pos=140, x_pos=-90)
paises = StaticText.static_text(canvas=canvas.canvas, text="Países",
                                 height=35, y_pos=140, x_pos=820)
canvas.add_sub_plot(pilotos)
canvas.add_sub_plot(paises)

In [28]:
canvas.play(fps=60)

FPS: 1
FPS: 32
FPS: 27
FPS: 23
FPS: 29
FPS: 34
FPS: 37
FPS: 23
FPS: 26
FPS: 38
FPS: 34
FPS: 34
FPS: 32
FPS: 33
FPS: 26
FPS: 32
FPS: 30
FPS: 34
FPS: 35
FPS: 36
FPS: 34
FPS: 27
FPS: 33
FPS: 33
FPS: 32
FPS: 33
FPS: 36
FPS: 32
FPS: 28
FPS: 33
FPS: 33
FPS: 33
FPS: 38
FPS: 35
FPS: 34
FPS: 38
FPS: 29
FPS: 34
FPS: 36
FPS: 37
FPS: 37
FPS: 36
FPS: 32
FPS: 30
FPS: 29
FPS: 33
FPS: 32
FPS: 32
FPS: 36
FPS: 34
FPS: 36
FPS: 32
FPS: 29
FPS: 33
FPS: 29
FPS: 33
FPS: 32
FPS: 29
FPS: 39
FPS: 28
FPS: 36
FPS: 34
FPS: 31
FPS: 33
FPS: 33
FPS: 33
FPS: 23
FPS: 29
FPS: 30
FPS: 31
FPS: 31
FPS: 31
FPS: 34
FPS: 28
FPS: 32
FPS: 30
FPS: 29
FPS: 34
FPS: 33
FPS: 30
FPS: 26
FPS: 34
FPS: 32
FPS: 29
FPS: 32
FPS: 31
FPS: 29
FPS: 28
FPS: 31
FPS: 33
FPS: 31
FPS: 31
FPS: 32
FPS: 37
FPS: 32
FPS: 23
FPS: 32
FPS: 31
FPS: 31
FPS: 31
FPS: 28
FPS: 24
FPS: 31
FPS: 37
FPS: 32
FPS: 32
FPS: 33
FPS: 26
FPS: 27
FPS: 32
FPS: 32
FPS: 32
FPS: 31
FPS: 31
FPS: 28
FPS: 29
FPS: 30
FPS: 31
FPS: 32
FPS: 33
FPS: 32
FPS: 27
FPS: 29
FPS: 29
FPS: 29
F

FPS: 25
FPS: 30
FPS: 27
FPS: 33
FPS: 29
FPS: 31
FPS: 29
FPS: 26
FPS: 29
FPS: 27
FPS: 27
FPS: 29
FPS: 26
FPS: 32
FPS: 23
FPS: 32
FPS: 29
FPS: 30
FPS: 30
FPS: 23
FPS: 31
FPS: 29
FPS: 30
FPS: 28
FPS: 29
FPS: 30
FPS: 23
FPS: 29
FPS: 29
FPS: 29
FPS: 31
FPS: 33
FPS: 30
FPS: 29
FPS: 30
FPS: 33
FPS: 32
FPS: 31
FPS: 30
FPS: 32
FPS: 27
FPS: 29
FPS: 30
FPS: 29
FPS: 32
FPS: 32
FPS: 30
FPS: 29
FPS: 30
FPS: 32
FPS: 30
FPS: 32
FPS: 31
FPS: 31
